# LLM interaction with langchain

In [4]:
import os
import re
import sys
sys.path.append('../src/')

import json
import importlib
import chromadb
import pickle as pkl
import pandas as pd
from dotenv import dotenv_values
from langchain_chroma import Chroma
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_core.runnables import chain
from langchain.chat_models import ChatOpenAI
from langchain_voyageai import VoyageAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from utils.fuzzyretriever import FuzzyRetriever

In [5]:
for i,v in dotenv_values().items():
    globals()[i]=v 

In [3]:
RESOURCES_DIR="../../resources"
llm = init_chat_model("o4-mini", model_provider="openai")
# llm = ChatOllama(model="qwen3:8b")
embeddings_model = VoyageAIEmbeddings(model="voyage-3")

Texto a probar

In [ ]:
test_df = pd.read_csv(os.path.join(PROJECT_DIR, "./datasets/TFM_test.csv"))
test_df.annotations = test_df.annotations.apply(eval)
clinical_note = test_df.texts.loc[1]
hpo_codes = test_df.annotations.iloc[1]
clinical_note

"Un hombre de 32 años acudió a una unidad regional de cirugía general por anemia sintomática grave. En un lapso de 4 meses, su hemoglobina descendió 62 puntos desde 137 gL (N: 135-180 gL). Su historial médico incluía epilepsia con valproato, arritmia cardiaca con flecainida, síndrome del túnel carpiano y compresión de la raíz nerviosa C5. No fuma, consume poco alcohol, tiene dos perros en casa y dos hijos. Sus antecedentes familiares incluían a su abuelo, que tuvo cáncer de intestino diagnosticado a los 78 años, y una abuela con cáncer de mama. La primera modalidad de diagnóstico por imagen fue un TAC que mostraba una gran masa fungiforme en el cuerpo del estómago que se extendía hasta el píloro y la primera parte del duodeno. Se sometió a una endoscopia digestiva alta (EDA) para investigar la patología. La histopatología arrojó con frecuencia un resultado de tejido hiperplásico benigno. Los primeros resultados histopatológicos informaron de mucosa gástrica con hiperplasia foveolar men

## Load retrievers

In [4]:
chroma_client = chromadb.HttpClient(host='localhost', port=8001)

In [5]:
vectordb = Chroma(client=chroma_client, embedding_function=embeddings_model, 
                  collection_name="hpo_ontology_esp_FULL")
retriever = vectordb.as_retriever(search_kwargs={"k": 20})

In [ ]:
with open(os.path.join(PROJECT_DIR, "resources", "keyword_retriever.pkl"), "rb") as fp:
    keyword_retriever = pkl.load(fp)

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectordb.as_retriever(),
                                                   keyword_retriever],
                                       weights=[0.6, 0.4], id_key="hpo_id", k=4)

In [ ]:
fuzzyretriever = FuzzyRetriever()

## First LLM call

In [15]:
system =  """Eres un experto de codificación de fenotipos de la ontología Human Phenotype Ontology. Para ello primero debes determinar qué fenotipos están presentes en la nota clínica. Sigue los siguientes pasos: 
1. A partir del siguiente texto clínico, identifica términos del texto que sugieran fenotipos clínicos relevantes, incluyendo diagnósticos, síntomas, signos físicos, hallazgos de laboratorio y modos de herencia.  
2. Si algún valor incluye de forma implícita un fenotipo, infiérelo y menciónalo como tal en el campo "phenotype".
3. Si el valor no permite inferir con seguridad un fenotipo, simplemente describe el resultado de la analítica en lenguaje natural.
4. Para cada término (extract), a parte del fenotipo, incluye la frase a la que pertenezca en la nota clínica original (context).
5. Sé específico, cada término debe contener un solo fenotipo asociado. Si tiene dos fenotipos, duplícalo y menciona ambos fenotipos. 
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{clinical_note}"),
    ]
)

In [7]:
class PhenotypeCandidate(BaseModel):
    """Fenotipos, patrones de herencia genética, anomalías anatómicas, síntomas clínicos, hallazgos diagnósticos, resultados de pruebas y afecciones o síndromes específicos en la nota clínica"""
    extract: str = Field(description="Un único fenotipo, diagnóstico, sintoma clínico, anomalia anatómica o prueba de laboratorio")
    phenotype: str = Field(description="Nombre del posible fenotipo asociado en español")
    context: str = Field( description="Deja esto vacío")


class Data(BaseModel):
    """Información extraída sobre los fenotipos encontrados y los términos y contexto en el que se encuentran"""
    candidates: List[PhenotypeCandidate]

In [16]:
structured_llm = llm.with_structured_output(Data)

In [ ]:
extractphenotypes = prompt | structured_llm
answer = extractphenotypes.invoke(
    {
        "clinical_note": clinical_note
    }
)

## Second LLM Call

In [ ]:
sys_template = """Identifica el término de la Ontología de Fenotipos Humanos (HPO) más apropiado para cada extracto de las notas clínicas del paciente a partir de una lista de candidatos (Código HPO - Descripción).
Da prioridad a los términos que sean concisos y directamente pertinentes para el síntoma o afección principal descritos. 
Céntrate en el tema central de cada frase y evita seleccionar opciones con detalles descriptivos o situacionales adicionales a menos que sean esenciales para captar con precisión el fenotipo. 
Asegúrate de que el término HPO elegido coincide estrechamente con la afección del paciente tal como se describe, sin añadir términos nuevos o extraños. 
Si hay varios candidatos, selecciona y devuelve el término HPO más pertinente que mejor represente la afección o síntoma primario. Proporciona sólo los códigos HPO elegidos bajo la llave 'hpo_code' y con el formato 'HPXXXXXXX' donde la X es un número. La nota clínica original es la siguiente:
{clinical_note}
"""

human_template =  """Término: {term} ({phenotype})
Contexto: {context}
Candidatos: {candidates}"""

chat_template = ChatPromptTemplate.from_messages([
    ('system', sys_template),
    ('human', human_template)
])

# mesages = chat_template.format_messages(term="Bob", context="What is your name?", candidates="hey")

In [ ]:
class HpoCode(BaseModel):
    """Código HPO asignado con el formato HP:#######"""
    hpo_code: str

In [ ]:
llm_output = llm.with_structured_output(HpoCode)
hpo_assignment = chat_template | llm_output

## Full chain

In [ ]:
def pretty_print_candidates(docs):
    final_str = ""
    for doc in docs: 
        final_str += f"{doc.id} - {doc.page_content}\n"
    return final_str

In [ ]:
@chain
def custom_chain(question):
    response = extractphenotypes.invoke(question)
    docs = []
    intermediate_results = []
    for query in response.candidates:
        new_docs = retriever.invoke(query.extract)
        fuzzy_docs = [x[0] for x in fuzzyretriever.invoke(query.phenotype)]
        new_docs =  vectordb.get_by_ids(set(fuzzy_docs)) + new_docs if len(set(fuzzy_docs)) > 0 else new_docs
        docs.append({"clinical_note":question,
                     "term":query.extract,
                     "phenotype":query.phenotype,
                     "context":query.context,
                     "candidates":pretty_print_candidates(new_docs)})
        intermediate_results.append([doc.id for doc in new_docs])    
    answer = hpo_assignment.batch(docs)
    return {"final answer": answer, "docs": intermediate_results}

In [ ]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="pk-lf-4c4b9492-d7ca-4f6d-b7af-325ea09726c0",
    secret_key="sk-lf-1b2e42c8-bf1e-4150-bf36-fce439ad2270",
    host="http://localhost:3000"
)

In [ ]:
answer = custom_chain.with_config(
    {
        "run_name": "qwen",
        "metadata": {"version": "v1", "owner": "mdiazrio"},
        "callbacks": [langfuse_handler]
    }
).invoke({"clinical_note":clinical_note})

## Extra

### Raw GPT

In [ ]:
system = """Eres una herramienta que sirve para extraer fenotipos de la ontología Human Phenotype Ontolgy a partir de notas clínicas para ello: 
1. A partir del siguiente texto clínico, identifica todos los fenotipos clínicos relevantes, incluyendo diagnósticos, síntomas, signos físicos y hallazgos de laboratorio. 
2. Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. 
3. Si algún valor incluye de forma implícita un fenotipo, infiérelo y menciónalo como tal.
4. Para cada término, asigna el código HPO apropiado. 
Devuelve un JSON con la llave "final_answer" y el listado de códigos HPO detectados.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{clinical_note}"),
    ]
)

In [ ]:
init_chain = prompt | llm

@chain
def rawgptchain(question):
    response = init_chain.invoke(question)
    return json.loads(response.content)

In [ ]:
response = rawgptchain.with_config({"callbacks": [langfuse_handler]}).invoke({"clinical_note":clinical_note})

### FlashRank

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from flashrank import Ranker 

In [ ]:
ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
reranked_docs = []
for context in [c.context for c in answer.candidates]:
    reranked_docs.append(compression_retriever.invoke(context)) 